In [93]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse, urljoin
import pandas as pd
import numpy as np
from datetime import date
import json
from datetime import date

In [94]:
today = date.today()

# Load Dataframe

In [95]:
df = pd.read_csv('JobsDataframe.csv')

known_job_ids = df.ID.unique()

# Jobscout24

In [96]:
main_url = 'https://www.jobscout24.ch/de/jobs/'

known_job_ids = df[df.JobsSite == 'Jobscout24'].ID.unique()

## Find Job URLs

In [97]:
def find_job_urls_on_page(url):
    
    job_urls = []
    
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    soup_articles = soup.find_all("li", {"class": "job-list-item"})
    page = url.split('=')[-1]
    
    for soup_article in soup_articles:

        job_dic = {}
        index_id = str(soup_article).index('data-job-id')
        data_id = str(soup_article)[index_id + len('data-job-id='):].split('"')[1]
        article_url = f'{main_url}?jobId={data_id}'
        job_dic['ArticleURL'] = article_url
        job_dic['Page'] = page
        job_dic['MainURL'] = url
        job_dic['ID'] = data_id
        job_urls.append(job_dic)
    
    
    return job_urls

In [98]:
def find_all_job_urls(url, pages):
    
    
    all_job_urls = []
    job_dic = {}
    for page in range(pages):
        next_page_url = f'{url}?p={page+1}'
        all_job_urls += find_job_urls_on_page(next_page_url)
        
    return all_job_urls

## Get URLs based on Search Terms

In [99]:
search_terms = [
    'Data Analyst',
    'Data Scientist',
    'Python Developer',
    'Python',
    'Data Engineer',
    'Business Intelligence',
    'Data Warehouse'
]

df_links = pd.DataFrame()

for search_term in search_terms:
    
    search_term_reworked = search_term.replace(' ', '%20').lower()
    search_term_url = f'{main_url}/{search_term_reworked}/'
    job_urls = find_all_job_urls(search_term_url, 5)
    df_new_links = pd.DataFrame(job_urls)
    df_new_links['SearchTerm'] = search_term
    df_links = pd.concat([df_links, df_new_links], ignore_index = True)

df_links['Date'] = today

In [100]:
df_count_ids = df_links.ID.value_counts().reset_index()
df_count_ids.columns = ['ID', 'NumberOfOccurences']

df_links_extended = df_links.merge(df_count_ids, on = 'ID', how = 'left')

df_links_extended.Page = df_links_extended.Page.astype('int64')
df_links_extended.ID = df_links_extended.ID.astype('int64')
df_links_extended.NumberOfOccurences = df_links_extended.NumberOfOccurences.astype('int64')

df_links_extended.Date = pd.to_datetime(df_links_extended.Date)

## Get Content from URL

In [101]:
def get_job_content(url):
    
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    job_description_soup = soup.find('div', {'class': 'slim_text'})
    job_title_soup = soup.find('div', {'class': 'slim_title'})
    company_title_soup = soup.find('h2', {'class': 'company-title'})
    company_location_soup = soup.find('a', {'class': 'company-location'})
    
    try:
        job_description = job_description_soup.get_text()
    except:
        job_description = 'Cannot access url'
        
    try:
        job_title = job_title_soup.get_text()
    except:
        job_title = 'Cannot access url'
        
    try:
        company_title = company_title_soup.get_text()
    except:
        company_title = 'Cannot access url'   
        
    try:
        company_location = company_location_soup.get_text()
    except:
        company_location = 'Cannot access url'
        
    
    return job_description, job_title, company_title, company_location

In [102]:
df_new = df_links_extended[~df_links_extended.ID.isin(known_job_ids)].copy()
df_new = df_new.drop_duplicates(subset='ID')
df_new = df_new.reset_index(drop=True)

df_new.columns = ['ArticleURL', 'Page', 'MainURL', 'ID', 'SearchTerm', 'FirstDate',
       'NumberOfOccurences']

df_new['LatestDate'] = df_new.FirstDate

In [103]:
len(df_new)

602

In [104]:
df_new_job_data = pd.DataFrame(df_new.ArticleURL.map(lambda url: get_job_content(url)).tolist())

df_new['JobDescription'] = df_new_job_data[0]
df_new['JobTitle'] = df_new_job_data[1]
df_new['CompanyName'] = df_new_job_data[2]
df_new['CompanyLocation'] = df_new_job_data[3]
df_new['JobsSite'] = 'Jobscout24'

# Data Mining

## With Job Title

In [105]:
def data_mining_job_title(dataframe):
    dataframe['JobDescriptionLength'] = dataframe.JobTitle.map(lambda x: len(x))
    dataframe['OneHotDataAnalyst'] = dataframe.JobTitle.str.lower().str.contains('data analyst').astype(int)
    dataframe['OneHotDataScientist'] = dataframe.JobTitle.str.lower().str.contains('data scientist').astype(int)
    dataframe['OneHotPython'] = dataframe.JobTitle.str.lower().str.contains('python').astype(int)
    dataframe['OneHotDeveloper'] = dataframe.JobTitle.str.lower().str.contains('developer').astype(int)
    dataframe['OneHotEngineer'] = dataframe.JobTitle.str.lower().str.contains('engineer').astype(int)
    dataframe['OneHotMathematik'] = dataframe.JobTitle.str.lower().str.contains('mathematik').astype(int)
    dataframe['OneHotData'] = dataframe.JobTitle.str.lower().str.contains('data').astype(int)
    dataframe['OneHotSenior'] = dataframe.JobTitle.str.lower().str.contains('senior').astype(int)
    return dataframe

In [106]:
df_new = data_mining_job_title(df_new)

## With Job Description

In [107]:
def data_mining_job_description(dataframe):
    dataframe['JobDescriptionLength'] = dataframe.JobDescription.map(lambda x: len(x))
    dataframe['CountDashboard'] = dataframe.JobDescription.str.lower().str.contains('dashboard').astype(int)
    dataframe['CountKomplex'] = dataframe.JobDescription.str.lower().str.contains('komplex').astype(int)
    dataframe['CountPython'] = dataframe.JobDescription.str.lower().str.contains('python').astype(int)
    dataframe['CountJavascript'] = dataframe.JobDescription.str.lower().str.contains('javascript').astype(int)
    dataframe['CountAnalytic'] = dataframe.JobDescription.str.lower().str.contains('analytic').astype(int)
    dataframe['CountModel'] = dataframe.JobDescription.str.lower().str.contains('model').astype(int)
    dataframe['CountMathematik'] = dataframe.JobDescription.str.lower().str.contains('mathematik').astype(int)
    dataframe['CountData'] = dataframe.JobDescription.str.lower().str.contains('data').astype(int)
    dataframe['CountHomeoffice'] = dataframe.JobDescription.str.lower().str.contains('homeoffice').astype(int)
    dataframe['CountFlexibel'] = dataframe.JobDescription.str.lower().str.contains('flexibel').astype(int)
    return dataframe

In [108]:
df_new = data_mining_job_description(df_new)

In [109]:
df = pd.concat([df, df_new], ignore_index = True)
df = df.sort_values('NumberOfOccurences', ascending = False)

/var/folders/dv/942svhyn703_lgsm3p02fyk40000gn/T/ipykernel_27120/4206080667.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_new], ignore_index = True)


# Update Dates of Old Job Data

In [110]:
df_known_job_ids = df_links_extended[df_links_extended.ID.isin(known_job_ids)].copy()
for index, row in df_known_job_ids.iterrows():
    job_id = int(row.ID)
    df.loc[df.ID == job_id, 'LatestDate'] = today

df.FirstDate = pd.to_datetime(df.FirstDate)
df.LatestDate = pd.to_datetime(df.LatestDate)

# Create Rankings

## Data Ranking

In [111]:
def create_data_ranking(row):
    return row.OneHotDataAnalyst + row.OneHotDataScientist + row.OneHotPython + row.OneHotDeveloper + row.OneHotEngineer + \
    row.OneHotMathematik + row.OneHotData + row.CountDashboard + row.CountKomplex + row.CountPython + row.CountAnalytic + \
    row.CountModel + row.CountMathematik + row.CountData + row.CountHomeoffice + row.CountFlexibel - 2 * row.OneHotSenior

In [112]:
df['DataRanking'] = df.apply(lambda row: create_data_ranking(row), axis=1)

## Developer Ranking

In [113]:
def create_developer_ranking(row):
    return row.OneHotPython + row.OneHotDeveloper + row.OneHotEngineer + \
    row.CountKomplex + row.CountPython + row.CountAnalytic + row.CountJavascript + \
    row.CountModel + row.CountData + row.CountHomeoffice + row.CountFlexibel - 2 * row.OneHotSenior

In [114]:
df['DeveloperRanking'] = df.apply(lambda row: create_developer_ranking(row), axis=1)

## Python Ranking

In [115]:
def create_python_ranking(row):
    if row.CountPython == 1:
        count_python = row.JobDescription.lower().count('python')
        return row.OneHotPython + row.OneHotDeveloper + row.CountHomeoffice + row.CountFlexibel + count_python
    else:
        return 0

In [116]:
df['PythonRanking'] = df.apply(lambda row: create_python_ranking(row), axis=1)

In [117]:
df = df.sort_values('DataRanking', ascending = False)

df = df.drop_duplicates('ID')
df = df[df.LatestDate == pd.to_datetime(today)].copy()
df = df[df.JobTitle != 'Cannot access url']
df = df.reset_index(drop=True)

# Auswertung

## Funktionen

In [118]:
def make_clickable(val):
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

In [119]:
def get_top30(df_new, ranking):

    df_new = df_new.sort_values(ranking, ascending = False)

    df_links = df_new.filter(['ArticleURL', 'JobTitle', ranking, 'CompanyName', 'CompanyLocation', 'SearchTerm', 'LatestDate', 'Page', 'FirstDate'])
    
    df_job_watching = df_links.head(n=30).style.format({'ArticleURL': make_clickable})
    
    return df_job_watching

In [120]:
def get_top10_new(df_new, ranking):

    df_new = df_new.sort_values(ranking, ascending = False)
    
    newest_date = df.FirstDate.sort_values().iloc[-1]
    
    df_new = df_new[df_new.FirstDate == newest_date]

    df_links = df_new.filter(['ArticleURL', 'JobTitle', ranking, 'CompanyName', 'CompanyLocation', 'SearchTerm', 'LatestDate', 'Page', 'FirstDate'])
    
    df_job_watching = df_links.head(n=10).style.format({'ArticleURL': make_clickable})
    
    return df_job_watching

## Top 10 neue Jobs

In [121]:
get_top10_new(df, 'PythonRanking')

,ArticleURL,Page,SearchTerm,FirstDate,LatestDate,JobTitle,PythonRanking,CompanyName,CompanyLocation
172,https://www.jobscout24.ch/de/jobs/?jobId=8218308,1,Python,2023-09-18 00:00:00,2023-09-18 00:00:00,Développeur Fullstack Python / Angular (H/F),4,Meanquest,Genf
21,https://www.jobscout24.ch/de/jobs/?jobId=8273421,1,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,Scientific Software Engineer,4,ETH Zürich,Zürich
66,https://www.jobscout24.ch/de/jobs/?jobId=8299836,4,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,R&D Praktikant (m/w/d) im Bereich optische Gassensoren,3,Sensirion AG,Stäfa
67,https://www.jobscout24.ch/de/jobs/?jobId=8370464,4,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,R&D Praktikant (m/w/d) im Bereich chemische Gassensoren,3,Sensirion AG,Stäfa
19,https://www.jobscout24.ch/de/jobs/?jobId=8137594,4,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,R&D Praktikant (m/w/d) im Bereich Process Engineering Flow Sensor Kalibration,3,Sensirion AG,Stäfa
73,https://www.jobscout24.ch/de/jobs/?jobId=8368518,4,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,R&D Praktikant (m/w/d) im Bereich Feuchte- und Temperatursensoren,3,Sensirion AG,Stäfa
132,https://www.jobscout24.ch/de/jobs/?jobId=8337114,1,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,Scientific Software Developer 50-80%,3,ictjobs (Stellenmarkt),Cannot access url
27,https://www.jobscout24.ch/de/jobs/?jobId=8328523,2,Python,2023-09-18 00:00:00,2023-09-18 00:00:00,System-Engineer:in (m/w/d),3,ETH Zürich,Zürich
65,https://www.jobscout24.ch/de/jobs/?jobId=8334066,3,Python,2023-09-18 00:00:00,2023-09-18 00:00:00,Innovation Fellow 80%-100%,2,ictjobs (Stellenmarkt),Bern
55,https://www.jobscout24.ch/de/jobs/?jobId=8369398,1,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,Electronics and Embedded Software developer for medical applications,2,Helbling Technik,Cannot access url


In [122]:
get_top10_new(df, 'DataRanking')

,ArticleURL,Page,SearchTerm,FirstDate,LatestDate,JobTitle,DataRanking,CompanyName,CompanyLocation
0,https://www.jobscout24.ch/de/jobs/?jobId=8338696,4,Data Analyst,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Analyst 100%,8,Office World Group,Cannot access url
2,https://www.jobscout24.ch/de/jobs/?jobId=8381922,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Scientist (m/w/d),7,Hays (Schweiz) AG,Zürich
3,https://www.jobscout24.ch/de/jobs/?jobId=8260455,1,Data Engineer,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Architect / Full-Stack Software Engineer,7,ETH Zürich,Schlieren
4,https://www.jobscout24.ch/de/jobs/?jobId=8186749,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,IoT Data Scientist (W/M),7,BSA - Bobst Mex SA,Cannot access url
5,https://www.jobscout24.ch/de/jobs/?jobId=8301487,2,Data Analyst,2023-09-18 00:00:00,2023-09-18 00:00:00,Junior Data Analyst 80 - 100%,7,yellowshark® AG,Baden
1,https://www.jobscout24.ch/de/jobs/?jobId=8366854,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Scientist 100%,7,Viseca Payment Services SA,Zürich
6,https://www.jobscout24.ch/de/jobs/?jobId=8387904,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,IoT Data Scientist (a),6,Lionstep AG,Genf
7,https://www.jobscout24.ch/de/jobs/?jobId=8388008,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Scientist (a),6,Lionstep AG,Lausanne
8,https://www.jobscout24.ch/de/jobs/?jobId=8272014,1,Data Analyst,2023-09-18 00:00:00,2023-09-18 00:00:00,Junior Data Analyst (f|m|d) (100%) - Zurich - Hybrid Work,6,SMG Swiss Marketplace Group AG,Zürich
9,https://www.jobscout24.ch/de/jobs/?jobId=7762743,4,Data Analyst,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Analyst,6,Business & Decision (Suisse) SA,Genf


In [123]:
get_top10_new(df, 'DeveloperRanking')

,ArticleURL,Page,SearchTerm,FirstDate,LatestDate,JobTitle,DeveloperRanking,CompanyName,CompanyLocation
3,https://www.jobscout24.ch/de/jobs/?jobId=8260455,1,Data Engineer,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Architect / Full-Stack Software Engineer,7,ETH Zürich,Schlieren
0,https://www.jobscout24.ch/de/jobs/?jobId=8338696,4,Data Analyst,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Analyst 100%,5,Office World Group,Cannot access url
10,https://www.jobscout24.ch/de/jobs/?jobId=8359060,1,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,Data-Architect:in / BI-Entwickler:in,5,Krebsliga Schweiz,Bern
16,https://www.jobscout24.ch/de/jobs/?jobId=8305595,2,Data Engineer,2023-09-18 00:00:00,2023-09-18 00:00:00,CPG Process Engineer at the Geothermal Energy and Geofluids Group,5,ETH Zürich,Zürich
21,https://www.jobscout24.ch/de/jobs/?jobId=8273421,1,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,Scientific Software Engineer,5,ETH Zürich,Zürich
1,https://www.jobscout24.ch/de/jobs/?jobId=8366854,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Scientist 100%,5,Viseca Payment Services SA,Zürich
11,https://www.jobscout24.ch/de/jobs/?jobId=8273953,2,Business Intelligence,2023-09-18 00:00:00,2023-09-18 00:00:00,IT-Business Engineer (m/w) 80-100%,5,yellowshark® AG,Zürich
19,https://www.jobscout24.ch/de/jobs/?jobId=8137594,4,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,R&D Praktikant (m/w/d) im Bereich Process Engineering Flow Sensor Kalibration,5,Sensirion AG,Stäfa
23,https://www.jobscout24.ch/de/jobs/?jobId=8280469,3,Data Warehouse,2023-09-18 00:00:00,2023-09-18 00:00:00,Consultant SAP Analytics,4,Lionstep AG,Bern
53,https://www.jobscout24.ch/de/jobs/?jobId=8353354,1,Python,2023-09-18 00:00:00,2023-09-18 00:00:00,Hochschul-Absolventen als Software Engineer (Java/Python) (m/w) 80-100%,4,yellowshark® AG,Zürich


## Top 30 Jobs

In [124]:
get_top30(df, 'PythonRanking')

,ArticleURL,Page,SearchTerm,FirstDate,LatestDate,JobTitle,PythonRanking,CompanyName,CompanyLocation
172,https://www.jobscout24.ch/de/jobs/?jobId=8218308,1,Python,2023-09-18 00:00:00,2023-09-18 00:00:00,Développeur Fullstack Python / Angular (H/F),4,Meanquest,Genf
21,https://www.jobscout24.ch/de/jobs/?jobId=8273421,1,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,Scientific Software Engineer,4,ETH Zürich,Zürich
66,https://www.jobscout24.ch/de/jobs/?jobId=8299836,4,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,R&D Praktikant (m/w/d) im Bereich optische Gassensoren,3,Sensirion AG,Stäfa
67,https://www.jobscout24.ch/de/jobs/?jobId=8370464,4,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,R&D Praktikant (m/w/d) im Bereich chemische Gassensoren,3,Sensirion AG,Stäfa
19,https://www.jobscout24.ch/de/jobs/?jobId=8137594,4,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,R&D Praktikant (m/w/d) im Bereich Process Engineering Flow Sensor Kalibration,3,Sensirion AG,Stäfa
73,https://www.jobscout24.ch/de/jobs/?jobId=8368518,4,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,R&D Praktikant (m/w/d) im Bereich Feuchte- und Temperatursensoren,3,Sensirion AG,Stäfa
132,https://www.jobscout24.ch/de/jobs/?jobId=8337114,1,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,Scientific Software Developer 50-80%,3,ictjobs (Stellenmarkt),Cannot access url
27,https://www.jobscout24.ch/de/jobs/?jobId=8328523,2,Python,2023-09-18 00:00:00,2023-09-18 00:00:00,System-Engineer:in (m/w/d),3,ETH Zürich,Zürich
65,https://www.jobscout24.ch/de/jobs/?jobId=8334066,3,Python,2023-09-18 00:00:00,2023-09-18 00:00:00,Innovation Fellow 80%-100%,2,ictjobs (Stellenmarkt),Bern
55,https://www.jobscout24.ch/de/jobs/?jobId=8369398,1,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,Electronics and Embedded Software developer for medical applications,2,Helbling Technik,Cannot access url


In [125]:
get_top30(df, 'DataRanking')

,ArticleURL,Page,SearchTerm,FirstDate,LatestDate,JobTitle,DataRanking,CompanyName,CompanyLocation
0,https://www.jobscout24.ch/de/jobs/?jobId=8338696,4,Data Analyst,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Analyst 100%,8,Office World Group,Cannot access url
2,https://www.jobscout24.ch/de/jobs/?jobId=8381922,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Scientist (m/w/d),7,Hays (Schweiz) AG,Zürich
3,https://www.jobscout24.ch/de/jobs/?jobId=8260455,1,Data Engineer,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Architect / Full-Stack Software Engineer,7,ETH Zürich,Schlieren
4,https://www.jobscout24.ch/de/jobs/?jobId=8186749,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,IoT Data Scientist (W/M),7,BSA - Bobst Mex SA,Cannot access url
5,https://www.jobscout24.ch/de/jobs/?jobId=8301487,2,Data Analyst,2023-09-18 00:00:00,2023-09-18 00:00:00,Junior Data Analyst 80 - 100%,7,yellowshark® AG,Baden
1,https://www.jobscout24.ch/de/jobs/?jobId=8366854,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Scientist 100%,7,Viseca Payment Services SA,Zürich
6,https://www.jobscout24.ch/de/jobs/?jobId=8387904,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,IoT Data Scientist (a),6,Lionstep AG,Genf
7,https://www.jobscout24.ch/de/jobs/?jobId=8388008,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Scientist (a),6,Lionstep AG,Lausanne
8,https://www.jobscout24.ch/de/jobs/?jobId=8272014,1,Data Analyst,2023-09-18 00:00:00,2023-09-18 00:00:00,Junior Data Analyst (f|m|d) (100%) - Zurich - Hybrid Work,6,SMG Swiss Marketplace Group AG,Zürich
9,https://www.jobscout24.ch/de/jobs/?jobId=7762743,4,Data Analyst,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Analyst,6,Business & Decision (Suisse) SA,Genf


In [126]:
get_top30(df, 'DeveloperRanking')

,ArticleURL,Page,SearchTerm,FirstDate,LatestDate,JobTitle,DeveloperRanking,CompanyName,CompanyLocation
3,https://www.jobscout24.ch/de/jobs/?jobId=8260455,1,Data Engineer,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Architect / Full-Stack Software Engineer,7,ETH Zürich,Schlieren
0,https://www.jobscout24.ch/de/jobs/?jobId=8338696,4,Data Analyst,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Analyst 100%,5,Office World Group,Cannot access url
10,https://www.jobscout24.ch/de/jobs/?jobId=8359060,1,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,Data-Architect:in / BI-Entwickler:in,5,Krebsliga Schweiz,Bern
16,https://www.jobscout24.ch/de/jobs/?jobId=8305595,2,Data Engineer,2023-09-18 00:00:00,2023-09-18 00:00:00,CPG Process Engineer at the Geothermal Energy and Geofluids Group,5,ETH Zürich,Zürich
21,https://www.jobscout24.ch/de/jobs/?jobId=8273421,1,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,Scientific Software Engineer,5,ETH Zürich,Zürich
1,https://www.jobscout24.ch/de/jobs/?jobId=8366854,1,Data Scientist,2023-09-18 00:00:00,2023-09-18 00:00:00,Data Scientist 100%,5,Viseca Payment Services SA,Zürich
11,https://www.jobscout24.ch/de/jobs/?jobId=8273953,2,Business Intelligence,2023-09-18 00:00:00,2023-09-18 00:00:00,IT-Business Engineer (m/w) 80-100%,5,yellowshark® AG,Zürich
19,https://www.jobscout24.ch/de/jobs/?jobId=8137594,4,Python Developer,2023-09-18 00:00:00,2023-09-18 00:00:00,R&D Praktikant (m/w/d) im Bereich Process Engineering Flow Sensor Kalibration,5,Sensirion AG,Stäfa
23,https://www.jobscout24.ch/de/jobs/?jobId=8280469,3,Data Warehouse,2023-09-18 00:00:00,2023-09-18 00:00:00,Consultant SAP Analytics,4,Lionstep AG,Bern
53,https://www.jobscout24.ch/de/jobs/?jobId=8353354,1,Python,2023-09-18 00:00:00,2023-09-18 00:00:00,Hochschul-Absolventen als Software Engineer (Java/Python) (m/w) 80-100%,4,yellowshark® AG,Zürich


# Export

In [127]:
df.to_csv('JobsDataframe.csv', index = False)